This notebook contains additional dataparsing steps that are taken after all the processing done by glp500 and JJ.

As input we use the excl sheets:
- bio_FINAL Julia working on sex.xlsx
- events_FINAL curated by Julia.xlsx
- locations(KEY) curated Julia.xlsx
- spouses_FINAL curated by Julia.xlsx

In [308]:
import pandas as pd
import re
from datetime import datetime

In [309]:
bio = pd.read_excel('./input_data/bio_FINAL Julia working on sex.xlsx')
events = pd.read_excel('./input_data/events_FINAL curated by Julia.xlsx')
locations = pd.read_excel('./input_data/locations(KEY) curated Julia.xlsx')
spouses = pd.read_excel('./input_data/spouses_FINAL curated by Julia.xlsx')


In [310]:
# Panda settings for showing data (this is foremost done to more easily explore the data while processing it)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In the bio table we are going to drop the origin column, since there are multiple errors in the translation (e.g. Austr. should be Australia and not Austria). Next we isolate the infomation about places (which are in Column Unnamed:6 ) and which are in between () e.g. (N) and (VS) and translate these into a new column with location (country) of birth.  

In [311]:
bio = bio.drop('origin', axis=1)

In [312]:
bio['oorsprong'] = bio['Unnamed: 6'].str.extract(r'\((.*?)\)')

Next, we want to replace the names of the country with a location file generated by JJ. We load the column as df after which we create a join between the two.  

In [313]:
locations.head()

,abr,location
0,(VS),Verenigde Staten
1,(N),Nederland
2,(Zwits.),Zwitserland
3,(Rusland),Rusland
4,(China),China


In [314]:
locations['abr'] = locations['abr'].str.replace(r'[()]', '', regex=True)

In [315]:
bio_location = pd.merge(bio, locations, left_on='oorsprong', right_on='abr', how='left')

Now we drop the two columns which are not needed anymore.

In [316]:
bio_location = bio_location.drop('oorsprong', axis=1)
bio_location = bio_location.drop('abr', axis=1)

For the sake of cleaning the data we convert the id to integer.

In [317]:
bio_location['id'] = bio_location['id'].astype('Int64')

In [318]:
bio_location.head()

,nama orang,geb/overl.,id,birthYear,deathYear,Unnamed: 6,Sex,location
0,"Aa, zr. W. van der",1908-1942/5,0,1908,1942/5,"Aa, zr. W. van der (N)",V,Nederland
1,"Aalbers, dr. Joh. Godefr., arts",1910-1992,1,1910,1992,"Aalbers, dr. Joh. Godefr., arts (N)",NaN,Nederland
2,"Aalders, zr. Jacoba",1910-1999,2,1910,1999,"Aalders, zr. Jacoba (N)",V,Nederland
3,"Abeel, David",1804-1846,3,1804,1846,"Abeel, David (VS)",M,Verenigde Staten
4,"Abkoude, ds. F.N.M. van",1895-1988,4,1895,1988,"Abkoude, ds. F.N.M. van (N)",M,Nederland


Now we extract the title from the column "nama orang" these are.

In [319]:
bio_location['titel'] = bio_location['nama orang'].str.extract(r'(dr\.|jhr\.|ds\.)', flags=re.IGNORECASE, expand=False)

Now remove the titles from the name. 

In [320]:
bio_location['titel'] = bio_location['nama orang'].str.extract(
    r'(dr\.|jhr\.|ds\.|drs.|jkvr.)', 
    flags=re.IGNORECASE, 
    expand=False
)

In [321]:
bio_location['nama orang'] = bio_location['nama orang'].str.replace(
    r'(dr\.|jhr\.|ds\.|drs.|jkvr.)\s*', 
    '', 
    flags=re.IGNORECASE, 
    regex=True
)

People have their profession ("beroep" in Dutch) in their name field. We extract this and parse it to a new field "beroep", since in some case people have multiple "beroepen" we add them into one column seperated by a ,

In [322]:
# Define the pattern of different professions (beroepen)
beroep_pattern = r'(zr\.|zr|arts|zdl\.|zendeling|evangel.|ond\.|zwm\.|zdl|diakones|diacones|theoloog)'

# Find the matches
bio_location['beroep(en)'] = bio_location['nama orang'].str.findall(beroep_pattern, flags=re.IGNORECASE)

# Join matches with commas (if any)
bio_location['beroep(en)'] = bio_location['beroep(en)'].apply(lambda x: ', '.join(x) if x else None)


Now we delete the professions from "nama orang"

In [323]:
bio_location['nama orang'] = bio_location['nama orang'].str.replace(
    r'(zr\.|zr|arts|zdl\.|verpl|zendeling|evangel.|ond\.|zwm\.|zwm|zdl|diakones|diacones|theoloog)\.?\s*',
    '',
    flags=re.IGNORECASE,
    regex=True
)

bio_location.head()

,nama orang,geb/overl.,id,birthYear,deathYear,Unnamed: 6,Sex,location,titel,beroep(en)
0,"Aa, W. van der",1908-1942/5,0,1908,1942/5,"Aa, zr. W. van der (N)",V,Nederland,NaN,zr.
1,"Aalbers, Joh. Godefr.,",1910-1992,1,1910,1992,"Aalbers, dr. Joh. Godefr., arts (N)",NaN,Nederland,dr.,arts
2,"Aalders, Jacoba",1910-1999,2,1910,1999,"Aalders, zr. Jacoba (N)",V,Nederland,NaN,zr.
3,"Abeel, David",1804-1846,3,1804,1846,"Abeel, David (VS)",M,Verenigde Staten,NaN,None
4,"Abkoude, F.N.M. van",1895-1988,4,1895,1988,"Abkoude, ds. F.N.M. van (N)",M,Nederland,ds.,None


In [324]:
bio_location.rename(columns={
    'birthYear': 'geboortejaar',
    'deathYear': 'sterfjaar',
    'Sex ': 'geslacht',
    'location' : 'oorsprong',
    'Unnamed: 6' : "original_input"     
}, inplace=True)

In [325]:
bio_location['titulatuur_geslacht'] = bio_location['nama orang'].str.extract(r'(mw\.| mw| ms.| ms)', flags=re.IGNORECASE, expand=False)

In [326]:
bio_location['nama orang'] = bio_location['nama orang'].str.replace(
    r'(mw\.|mw| ms.| ms)\.?\s*',
    '',
    flags=re.IGNORECASE,
    regex=True
)

In [327]:
bio_location.head(10)

,nama orang,geb/overl.,id,geboortejaar,sterfjaar,original_input,geslacht,oorsprong,titel,beroep(en),titulatuur_geslacht
0,"Aa, W. van der",1908-1942/5,0,1908,1942/5,"Aa, zr. W. van der (N)",V,Nederland,NaN,zr.,NaN
1,"Aalbers, Joh. Godefr.,",1910-1992,1,1910,1992,"Aalbers, dr. Joh. Godefr., arts (N)",NaN,Nederland,dr.,arts,NaN
2,"Aalders, Jacoba",1910-1999,2,1910,1999,"Aalders, zr. Jacoba (N)",V,Nederland,NaN,zr.,NaN
3,"Abeel, David",1804-1846,3,1804,1846,"Abeel, David (VS)",M,Verenigde Staten,NaN,None,NaN
4,"Abkoude, F.N.M. van",1895-1988,4,1895,1988,"Abkoude, ds. F.N.M. van (N)",M,Nederland,ds.,None,NaN
5,"Ackermann, Gustav,",1876-1955,5,1876,1955,"Ackermann, Gustav, zdl. (D)",M,Duitsland,NaN,zdl.,NaN
6,"Adriaanse, A.J., zie Gemerts-Adriaanse, A.J.",NaN,6,NaN,NaN,"Adriaanse, A.J., zie Gemerts-Adriaanse, A.J.",NaN,NaN,NaN,None,NaN
7,"Adriaanse, L.",1856-1947,7,1856,1947,"Adriaanse, ds. L. (N)",NaN,Nederland,ds.,None,NaN
8,"Adriani, Nicolaus",1865-1926,8,1865,1926,"Adriani, dr. Nicolaus (N)",M,Nederland,dr.,None,NaN
9,"Aebersold, W.E.,",1903-1996,9,1903,1996,"Aebersold, W.E., zdl (Zwits.)",NaN,Zwitserland,NaN,zdl,NaN


In [328]:
bio_location['achternaam'] = bio_location['nama orang'].str.extract(r'^([^,]+)')

In [329]:
bio_location['verwijzing'] = bio_location['nama orang'].str.extract(r'(zie\s+.*)', flags=re.IGNORECASE)
bio_location['nama orang'] = bio_location['nama orang'].str.replace(r'\s*zie\s+.*', '', flags=re.IGNORECASE, regex=True)

In [330]:
bio_location.head(25)

,nama orang,geb/overl.,id,geboortejaar,sterfjaar,original_input,geslacht,oorsprong,titel,beroep(en),titulatuur_geslacht,achternaam,verwijzing
0,"Aa, W. van der",1908-1942/5,0,1908,1942/5,"Aa, zr. W. van der (N)",V,Nederland,NaN,zr.,NaN,Aa,NaN
1,"Aalbers, Joh. Godefr.,",1910-1992,1,1910,1992,"Aalbers, dr. Joh. Godefr., arts (N)",NaN,Nederland,dr.,arts,NaN,Aalbers,NaN
2,"Aalders, Jacoba",1910-1999,2,1910,1999,"Aalders, zr. Jacoba (N)",V,Nederland,NaN,zr.,NaN,Aalders,NaN
3,"Abeel, David",1804-1846,3,1804,1846,"Abeel, David (VS)",M,Verenigde Staten,NaN,None,NaN,Abeel,NaN
4,"Abkoude, F.N.M. van",1895-1988,4,1895,1988,"Abkoude, ds. F.N.M. van (N)",M,Nederland,ds.,None,NaN,Abkoude,NaN
5,"Ackermann, Gustav,",1876-1955,5,1876,1955,"Ackermann, Gustav, zdl. (D)",M,Duitsland,NaN,zdl.,NaN,Ackermann,NaN
6,"Adriaanse, A.J.,",NaN,6,NaN,NaN,"Adriaanse, A.J., zie Gemerts-Adriaanse, A.J.",NaN,NaN,NaN,None,NaN,Adriaanse,"zie Gemerts-Adriaanse, A.J."
7,"Adriaanse, L.",1856-1947,7,1856,1947,"Adriaanse, ds. L. (N)",NaN,Nederland,ds.,None,NaN,Adriaanse,NaN
8,"Adriani, Nicolaus",1865-1926,8,1865,1926,"Adriani, dr. Nicolaus (N)",M,Nederland,dr.,None,NaN,Adriani,NaN
9,"Aebersold, W.E.,",1903-1996,9,1903,1996,"Aebersold, W.E., zdl (Zwits.)",NaN,Zwitserland,NaN,zdl,NaN,Aebersold,NaN


In [331]:
bio_location['infix'] = bio_location['nama orang'].str.extract(
    r'(van der|van de|van den|van|de)', 
    flags=re.IGNORECASE, 
    expand=False
)

bio_location['nama orang'] = bio_location['nama orang'].str.replace(
    r'\b(van der|van de|van den|van|de)\b\s*',
    '',
    flags=re.IGNORECASE,
    regex=True
)

In [332]:
bio_location['voornamen of voorletters'] = bio_location['nama orang'].str.extract(r',\s*(.*)')

In [333]:
bio_location['voornamen of voorletters'] = bio_location['voornamen of voorletters'].str.replace(',', '', regex=False)


In [334]:
bio_location.head(50)

,nama orang,geb/overl.,id,geboortejaar,sterfjaar,original_input,geslacht,oorsprong,titel,beroep(en),titulatuur_geslacht,achternaam,verwijzing,infix,voornamen of voorletters
0,"Aa, W.",1908-1942/5,0,1908,1942/5,"Aa, zr. W. van der (N)",V,Nederland,NaN,zr.,NaN,Aa,NaN,van der,W.
1,"Aalbers, Joh. Godefr.,",1910-1992,1,1910,1992,"Aalbers, dr. Joh. Godefr., arts (N)",NaN,Nederland,dr.,arts,NaN,Aalbers,NaN,de,Joh. Godefr.
2,"Aalders, Jacoba",1910-1999,2,1910,1999,"Aalders, zr. Jacoba (N)",V,Nederland,NaN,zr.,NaN,Aalders,NaN,de,Jacoba
3,"Abeel, David",1804-1846,3,1804,1846,"Abeel, David (VS)",M,Verenigde Staten,NaN,None,NaN,Abeel,NaN,NaN,David
4,"Abkoude, F.N.M.",1895-1988,4,1895,1988,"Abkoude, ds. F.N.M. van (N)",M,Nederland,ds.,None,NaN,Abkoude,NaN,de,F.N.M.
5,"Ackermann, Gustav,",1876-1955,5,1876,1955,"Ackermann, Gustav, zdl. (D)",M,Duitsland,NaN,zdl.,NaN,Ackermann,NaN,NaN,Gustav
6,"Adriaanse, A.J.,",NaN,6,NaN,NaN,"Adriaanse, A.J., zie Gemerts-Adriaanse, A.J.",NaN,NaN,NaN,None,NaN,Adriaanse,"zie Gemerts-Adriaanse, A.J.",NaN,A.J.
7,"Adriaanse, L.",1856-1947,7,1856,1947,"Adriaanse, ds. L. (N)",NaN,Nederland,ds.,None,NaN,Adriaanse,NaN,NaN,L.
8,"Adriani, Nicolaus",1865-1926,8,1865,1926,"Adriani, dr. Nicolaus (N)",M,Nederland,dr.,None,NaN,Adriani,NaN,NaN,Nicolaus
9,"Aebersold, W.E.,",1903-1996,9,1903,1996,"Aebersold, W.E., zdl (Zwits.)",NaN,Zwitserland,NaN,zdl,NaN,Aebersold,NaN,NaN,W.E.


In [335]:
# Extract the first 4-digit number from 'geboortejaar'
bio_location['geboortejaar_int'] = bio_location['geboortejaar'].astype(str).str.extract(r'(\d{4})')

# Convert to integer (optional, depending on if you want NaN or errors on failure)
bio_location['geboortejaar_int'] = bio_location['geboortejaar_int'].astype(float).astype('Int64')

In [336]:
# Extract the first 4-digit number from 'sterfjaar'
bio_location['sterfjaar_int'] = bio_location['sterfjaar'].astype(str).str.extract(r'(\d{4})')

# Convert to integer
bio_location['sterfjaar_int'] = bio_location['sterfjaar_int'].astype(float).astype('Int64')

In [337]:
bio_location['bron'] = None
bio_location['wildcard'] = None
bio_location['opmerkingen_wildcard'] = None
bio_location['verwzijzing_id'] = None


In [338]:
bio_location.columns.tolist()

['nama orang',
 'geb/overl.',
 'id',
 'geboortejaar',
 'sterfjaar',
 'original_input',
 'geslacht',
 'oorsprong',
 'titel',
 'beroep(en)',
 'titulatuur_geslacht',
 'achternaam',
 'verwijzing',
 'infix',
 'voornamen of voorletters',
 'geboortejaar_int',
 'sterfjaar_int',
 'bron',
 'wildcard',
 'opmerkingen_wildcard',
 'verwzijzing_id']

In [339]:
# change the order of the columns and drop "nama orang"

bio_location = bio_location[[
    'id', 
    'original_input', 
    'titel',
    'achternaam',
    'infix',
    'voornamen of voorletters',
    'beroep(en)',
    'titulatuur_geslacht',
    'geslacht',
    'oorsprong',
    'verwijzing',
    'verwzijzing_id',
    'geb/overl.',
    'geboortejaar',
    'geboortejaar_int',
    'sterfjaar',
    'sterfjaar_int',
    'bron',
    'wildcard',
    'opmerkingen_wildcard'
]]


In [340]:
date_str = datetime.today().strftime('%m_%d_%Y')
filename = f'bio_{date_str}.xlsx'

In [341]:
bio_location.to_excel(filename, index=False)